## Pre-processing

#### Import Libraries

In [1]:
import re
import pandas as pd
import numpy as np
from pathlib import Path
from janitor import clean_names

#### Derive cost centers from SAP for the GPA sub master
``` python
# Filenames
input_1 = path / "meta_cc" / "IMPR.xlsx"
input_2 = path / "meta_cc" / "IMZO.xlsx"
input_3 = path / "meta_cc" / "PRPS.xlsx"
output_file = path / "meta" / "cost_centers.csv"
```

In [2]:
!python "0_derive_cost_centers.py"

A files is created


## GPA

#### Process GPA spending data
``` python
# Filenames
input_file = path / "fc_data" / "2023-11_GPA_WMS - All data report_FC.xlsx"
output_file = path / "fc_output" / "fc_monthly_spending.csv"
```

In [3]:
!python "1_fc_GPA_spending.py"

A file is created


#### Process asset acquisition from GPA spending
``` python
input_file = path / "fc_output" / "fc_monthly_spending.csv"
output_file = path / "fc_output" / "fc_acquisition_future_assets.csv"
```

In [4]:
!python "2_fc_acquisition.py"

A file is created


#### Calculate the monthly deprecation of acquisitions of future assets from GPA
``` python
# Filenames
input_file = path / "fc_output" / "fc_acquisition_future_assets.csv"
output_file = path / "fc_output" / "fc_depreciation_future_assets.csv"
```
The key function for calculating monthly depreciation is as follows.
```python
def calc_monthly_depr(row):
    monthly_depr = row["acquisition"] / (row["useful_life_year"] * 12)
    return monthly_depr
```

In [5]:
!python "3_fc_depreciation.py"

A file is created


## SAP

#### Process asset acquisition from SAP asset list
``` python
# Filenames
input_file = path / "fc_data" / "2023-11_Asset History Leger_20231130.xlsx"
output_file = path / "fc_output" / "fc_acquisition_existing_assets.csv"
```

In [ ]:
!python "2_act_SAP_acquisition.py"

#### Calculate the monthly depreciation of acquisitions of existing assets from SAP asset list
``` python
# Filenames
input_file = path / "fc_output" / "fc_acquisition_existing_assets.csv"
output_file = path / "fc_output" / "fc_depreciation_existing_assets.csv"
```
##### Please wait, as this step takes around 15 seconds, The key function for calculating monthly depreciation is as follows.
``` python
def calc_monthly_depr(row, period_start, period_end):
    if (
        # To avoid Division by zero error (e.g. Asset under construction)
        row["useful_life_year"]
        == row["useful_life_month"]
        == 0
    ):
        return 0

    # Main calculation logic
    monthly_depr = row["acquisition"] / (
        row["useful_life_year"] * 12 + row["useful_life_month"]
    )
    alterative_monthly_depr = row["current"] * -1 / 12

    # If the depreciation starts after period_start
    if pd.to_datetime(period_start) < row["depr_start"]:
        date1 = pd.to_datetime(period_start)
        date2 = row["depr_start"]
        months_difference = (date2.year - date1.year) * 12 + date2.month - date1.month
        return row["current"] * -1 / (12 - months_difference)

    # If the depreciation ends between period_start and period_end
    elif pd.to_datetime(period_start) < row["depr_end"] and row[
        "depr_end"
    ] < pd.to_datetime(period_end):
        date1 = row["depr_end"]
        date2 = pd.to_datetime(period_end)
        months_difference = (date2.year - date1.year) * 12 + date2.month - date1.month
        return row["current"] * -1 / (12 - months_difference)

    # If it is still depreciated even after depr_end date because usage is restarted
    elif row["depr_end"] < pd.to_datetime(period_start) and abs(row["current"]) > 0:
        return alterative_monthly_depr

    # Yearly Depreciation is not equal to the expected value
    # due to additional investment, ICO asset transfer, etc
    elif abs(row["current"] + (monthly_depr * 12)) > 1:
        return alterative_monthly_depr

    else:
        return monthly_depr

```

In [6]:
!python "3_act_depreciation.py"

A file is created


## GPA + SAP

#### Combine the monthly depreciations of future assets from GPA and existing assets from SAP
``` python
# Filenames
input_1 = path / "fc_output" / "fc_depreciation_future_assets.csv"
input_2 = path / "fc_output" / "fc_depreciation_existing_assets.csv"
output_file = path / "fc_output" / "fc_depreciation_combined.csv"
```
##### And, we add "responsibilities" column with the following function.
``` python
def add_responsibilities(row):
    # define variable
    outlet_cf = ["Central Functions"]
    outlet_pl1 = ["PL ENC", "PL DTC", "PL MTC", "PL VBC"]
    outlet_pl2 = [
        "PL HVD",
        "PL EAC",
        "PL DAC",
        "PL MES",
        "PL HYD",
        "PL CM CCN",
        "PL CM CVS",
        "PL CM PSS",
    ]
    outlet_rnd = ["DIV E Eng.", "PL DAC", "PL MES"]
    outlet_shs = ["DIV E C", "DIV P C", "PT - Quality"]
    # If conditions
    if row["outlet_sender"] in outlet_cf:
        return "Central Functions"
    elif (row["location_receiver"] == "Icheon") & (row["outlet_sender"] in outlet_pl1):
        return "Productlines_1"
    elif (row["location_receiver"] == "Icheon") & (row["outlet_sender"] in outlet_pl2):
        return "Productlines_2"
    elif row["location_receiver"] == "Sejong":  # CM Inbound
        return "Productlines_2"
    elif (row["location_receiver"] == "Icheon NPF") & (
        row["outlet_sender"] in outlet_rnd
    ):
        return "R&D"
    elif (row["location_receiver"] == "Icheon NPF") & (
        row["outlet_sender"] in outlet_shs
    ):
        return "Share Service"
```

In [7]:
!python "4_combine_versions.py"

A file is created


#### Export GPA master and SAP Asset Under Construction for manual input
``` python
# Filenames
input_file = path / "fc_output" / "fc_depreciation_combined.csv"
output_master = path / "fc_output" / "fc_GPA_master.csv"
output_auc = path / "fc_output" / "fc_SAP_AUC.csv"
```

In [8]:
!python "4_export_additional_data.py"

Files are created
